In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 70.6 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB

# ========================================================
# Data
# ========================================================
# Points: each is [x, y, value]
crystals = [
    [10, 20, 100],
    [40, 50, 150],
    [25, 35, 200],
    [80, 50, 300]
]
mines = [
    [15, 25, 50],
    [70, 60, 75],
    [30, 200, 60]
]
n_crystals = len(crystals)
n_mines = len(mines)

In [4]:
import gurobipy as gp
from gurobipy import GRB

# ========================================================
# Data
# ========================================================
# Points: each is [x, y, value]
crystals = [
    [10, 20, 100],
    [40, 50, 150],
    [25, 35, 200],
    [50, 50, 300]
]
mines = [
    [15, 25, 50],
    [70, 60, 75],
    [30, 200, 60]
]
n_crystals = len(crystals)
n_mines = len(mines)

# ========================================================
# Model Parameters
# ========================================================
n_vertices = 20             # Reduced number of vertices for better convergence
big_M = 100000            # Increased Big-M constant
M_sep = 100000            # Increased separation constant
delta = 5.0               # Reduced minimum edge length
epsilon_convex = 1e-6     # Reduced epsilon for numerical stability

# Calculate center point based on data points
# center_x = sum(p[0] for p in crystals + mines) / (n_crystals + n_mines)
# center_y = sum(p[1] for p in crystals + mines) / (n_crystals + n_mines)

# ========================================================
# Create the Model
# ========================================================
params={
    "WLSACCESSID":'56123837-1b14-432c-b974-a9bb029df73e',
    "WLSSECRET":'ee7e4faf-676f-472b-b2f8-a196087f18d5',
    "LICENSEID":2616884
}
env=gp.Env(params=params)
model=gp.Model(env=env)

epsilon=1e-5

# Add MIPFocus parameter to emphasize finding feasible solutions
# model.setParam('MIPFocus', 1)
# model.setParam('NumericFocus', 3)  # Increase numeric focus for better stability

# --------------------------------------------------
# (A) Polygon Variables
# --------------------------------------------------
# Vertex coordinates (continuous)
x = model.addVars(n_vertices, lb=0, ub=250.0, name="x")
y = model.addVars(n_vertices, lb=0, ub=250.0, name="y")

# Binary variables for edge orientation
b = model.addVars(n_vertices, vtype=GRB.BINARY, name="b")
d = model.addVars(n_vertices, vtype=GRB.BINARY, name="d")

# --------------------------------------------------
# (B) Enforce Alternating Edge Orientation
# --------------------------------------------------
for i in range(n_vertices):
    model.addConstr(b[i] + b[(i+1) % n_vertices] == 1, name=f"alternating_{i}")

# --------------------------------------------------
# (C) Edge Non-Degeneracy and Minimum Length Constraints
# --------------------------------------------------
for i in range(n_vertices):
    i_next = (i + 1) % n_vertices

    # Horizontal/vertical constraints
    model.addConstr(y[i] - y[i_next] <= big_M * (1 - b[i]), name=f"hor_y_upper_{i}")
    model.addConstr(y[i] - y[i_next] >= -big_M * (1 - b[i]), name=f"hor_y_lower_{i}")
    model.addConstr(x[i] - x[i_next] <= big_M * b[i], name=f"ver_x_upper_{i}")
    model.addConstr(x[i] - x[i_next] >= -big_M * b[i], name=f"ver_x_lower_{i}")

    # Minimum length constraints with direction
    model.addConstr(x[i_next] - x[i] >= delta - big_M*(1 - d[i]) - big_M*(1 - b[i]))
    model.addConstr(x[i] - x[i_next] >= delta - big_M*d[i] - big_M*(1 - b[i]))
    model.addConstr(y[i_next] - y[i] >= delta - big_M*(1 - d[i]) - big_M*b[i])
    model.addConstr(y[i] - y[i_next] >= delta - big_M*d[i] - big_M*b[i])

# --------------------------------------------------
# (D) Non-Intersection Constraints
# --------------------------------------------------
M=1e6
for i in range(n_vertices):
  for j in range(i+2,n_vertices):
    if((i%2)!=(j%2)):
      i_next=(i+1)%n_vertices
      j_next=(j+1)%n_vertices

      # When b==1, enforce constraint1

      #b==1 indicates that i edge is horizontal constraint 1
      i_hor_sum = model.addVar(vtype=GRB.INTEGER,name=f"i_hor_sum{i}_{j}")

      xi_hor_min=model.addVar()
      xi_hor_max=model.addVar()
      model.addGenConstrMin(xi_hor_min, [x[i], x[i_next]], name="min_constraint")
      model.addGenConstrMax(xi_hor_max, [x[i], x[i_next]], name="max_constraint")

      yj_ver_min=model.addVar()
      yj_ver_max=model.addVar()
      model.addGenConstrMin(yj_ver_min, [y[j], y[j_next]], name="min_constraint")
      model.addGenConstrMax(yj_ver_max, [y[j], y[j_next]], name="max_constraint")

      hor1=model.addVar(vtype=GRB.BINARY)
      hor2=model.addVar(vtype=GRB.BINARY)
      hor3=model.addVar(vtype=GRB.BINARY)
      hor4=model.addVar(vtype=GRB.BINARY)

      model.addGenConstrIndicator(hor1,1,x[j]>=epsilon+xi_hor_max, name=f"if_case{i}_{j}")
      model.addGenConstrIndicator(hor2,1,x[j]<=xi_hor_min-epsilon, name=f"if_case{i}_{j}")
      model.addGenConstrIndicator(hor3,1,y[i]>=epsilon+yj_ver_max, name=f"if_case{i}_{j}")
      model.addGenConstrIndicator(hor4,1,y[i]<=yj_ver_min-epsilon, name=f"if_case{i}_{j}")

      model.addConstr(i_hor_sum==hor1+hor2+hor3+hor4)

      model.addGenConstrIndicator(b[i],1,i_hor_sum>=1, name=f"if_case{i}_{j}")





      i_ver_sum = model.addVar(vtype=GRB.INTEGER,name=f"i_hor_sum{i}_{j}")

      yi_ver_min=model.addVar()
      yi_ver_max=model.addVar()
      model.addGenConstrMin(yi_ver_min, [y[i], y[i_next]], name="min_constraint")
      model.addGenConstrMax(yi_ver_max, [y[i], y[i_next]], name="max_constraint")

      xj_hor_min=model.addVar()
      xj_hor_max=model.addVar()
      model.addGenConstrMin(xj_hor_min, [x[j], x[j_next]], name="min_constraint")
      model.addGenConstrMax(xj_hor_max, [x[j], x[j_next]], name="max_constraint")

      ver1=model.addVar(vtype=GRB.BINARY)
      ver2=model.addVar(vtype=GRB.BINARY)
      ver3=model.addVar(vtype=GRB.BINARY)
      ver4=model.addVar(vtype=GRB.BINARY)

      model.addGenConstrIndicator(ver1,1,x[i]<=xj_hor_min-epsilon, name=f"if_case{i}_{j}")
      model.addGenConstrIndicator(ver2,1,x[i]>=epsilon+xj_hor_max, name=f"if_case{i}_{j}")
      model.addGenConstrIndicator(ver3,1,y[j]>=epsilon+yi_ver_max, name=f"if_case{i}_{j}")
      model.addGenConstrIndicator(ver4,1,y[j]<=yi_ver_min-epsilon, name=f"if_case{i}_{j}")

      model.addConstr(i_ver_sum==ver1+ver2+ver3+ver4)
      model.addGenConstrIndicator(b[i],0,i_ver_sum>=1, name=f"if_case{i}_{j}")




# --------------------------------------------------
# (E) Point-Inclusion Constraints (Cross Product Method)
# --------------------------------------------------
inside_vars = []
for p, (xp, yp, _) in enumerate(crystals + mines):
    inside = model.addVar(vtype=GRB.BINARY, name=f"inside_{p}")
    inside_vars.append(inside)
    cross_vars = model.addVars(n_vertices, vtype=GRB.BINARY, name=f"cross_{p}")

    for i in range(n_vertices):
        i_next = (i + 1) % n_vertices
        cross_expr = (x[i_next] - x[i]) * (yp - y[i]) - (y[i_next] - y[i]) * (xp - x[i])
        model.addConstr(cross_expr >= -epsilon_convex - big_M * (1 - cross_vars[i]))
        model.addConstr(cross_expr <= epsilon_convex + big_M * (1 - cross_vars[i]))

    model.addConstr(inside <= gp.quicksum(cross_vars[i] for i in range(n_vertices)))
    model.addConstr(inside >= gp.quicksum(cross_vars[i] for i in range(n_vertices)) - n_vertices + 1)

# --------------------------------------------------
# (F) Objective
# --------------------------------------------------
objective_expr = gp.quicksum(crystals[p][2] * inside_vars[p] for p in range(n_crystals)) \
               - gp.quicksum(mines[p][2] * inside_vars[p + n_crystals] for p in range(n_mines))
model.setObjective(0, GRB.MAXIMIZE)

# --------------------------------------------------
# Solve the Model
# --------------------------------------------------
model.update()
model.optimize()

# --------------------------------------------------
# Report Results
# --------------------------------------------------
if model.status == GRB.OPTIMAL:
    print("\nOptimal Polygon Vertex Coordinates:")
    for i in range(n_vertices):
        print(f" Vertex {i}: (x = {x[i].X}, y = {y[i].X})")
    print("\nTotal Score =", model.ObjVal)
else:
    print(f"Optimization ended with status {model.status}")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2616884
Academic license 2616884 - for non-commercial use only - registered to ak___@iitg.ac.in
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license 2616884 - for non-commercial use only - registered to ak___@iitg.ac.in
Optimize a model with 356 rows, 1685 columns and 1704 nonzeros
Model fingerprint: 0x68ea08d2
Model has 280 quadratic constraints
Model has 1458 simple general constraints
  324 MAX, 324 MIN, 810 INDICATOR
Variable types: 688 continuous, 997 integer (835 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+01, 1e+05]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+05]
  QRHS range       [1e+05